In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path

# settings for making nice pdfs
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
base_folder_string = 'output_0712_tiling600_LonglivedLEFs_ProSepScan'
folders = [foldername for foldername in os.listdir('Data/') if foldername.startswith(base_folder_string)]

base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir = folders[m]
    print(data_dir)
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    sep_numerical_sorted = [float(a) for a in sep_list_sorted]
    proc_list = list(set([stats_file[i].split('_proc')[1].split('_superportion')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_list_sorted = [float(a) for a in proc_list_sorted]
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(proc_list_sorted),len(sep_list_sorted),len(bs_list_sorted)))
        ctcf_bs_success =   np.zeros((len(folders),len(proc_list_sorted),len(sep_list_sorted),len(bs_list_sorted)))
        ctcf_bs_realtime =  np.zeros((len(folders),len(proc_list_sorted),len(sep_list_sorted),len(bs_list_sorted)))
        ctcf_bs_time0 =    np.zeros((len(folders),len(proc_list_sorted),len(sep_list_sorted),len(bs_list_sorted)))
        ctcf_bs_time0_predicted =    np.zeros((len(folders),len(proc_list_sorted),len(sep_list_sorted),len(bs_list_sorted)))
    for i in range(len(proc_list_sorted)):
        for j in range(len(sep_list_sorted)):
            for k in range(len(bs_list_sorted)):
                if proc_numerical_list_sorted[i]==62.5:
                    par_combo = 'sep' + sep_list_sorted[j] + '_proc' + proc_list_sorted[i] + '_superportion' + superportion_list_sorted[0]+ '_superproc' + superproc_list_sorted[0] + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0] + '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[k]
                else:
                    par_combo = 'sep' + sep_list_sorted[j] + '_proc' + proc_list_sorted[i] + '_superportion' + superportion_list_sorted[0]+ '_superproc' + str(int(proc_numerical_list_sorted[i]*20)) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0] + '_superloading' + superloading_list_sorted[0]+ '_bs' + bs_list_sorted[k]

                filename = 'Data/'+data_dir + '/stats_df_' + par_combo + '.csv'
                if path.exists(filename):
                    stats_df = pd.read_csv(filename)
                    stats_df = stats_df.drop(columns='Unnamed: 0')
                    stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                    sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()


                    portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
                    portion_df = portion_df.drop(columns='Unnamed: 0')
                    portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                    ctcf_bs_time0[m,i,j,k] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]* 100
                    ctcf_bs_realtime[m,i,j,k] = portion_df_threshold1['restrained proportion realtime'].to_numpy()[0]
                    ctcf_bs_success[m,i,j,k] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100

                    proc = proc_numerical_list_sorted[i]
                    sep = sep_numerical_sorted[j]
                    x = proc/sep
                    TAD_list=np.asarray([200,400,800,1200])
                    weight = np.asarray([0.5,0.25,0.125,0.125])
                    
                    L_sep_ratio = 10**(-0.08238 + 0.7258*np.log10(x) - 
                                 0.2514*(np.log10(x))**2 - 
                                 0.003995*(np.log10(x))**3 + 
                                 0.03445*(np.log10(x))**4 - 
                                 0.01077*(np.log10(x))**5 + 
                                 0.001371*(np.log10(x))**6 - 
                                 6.472*10**(-5)*(np.log10(x))**7)
                    L= L_sep_ratio * sep
                    BS = bs_numerical_sorted[k]
                    
                    for o in range(len(TAD_list)):
                        TAD = TAD_list[o]
                        E0 = 1/TAD
                        S0 = 1/sep
                        D= TAD

                        if BS!=0:
                            a0= (1/D+1/sep+2/BS/proc-np.sqrt((1/D+1/sep+2/BS/proc)**2-4/D/sep))/2
                        else:
                            a0=0
                        c = a0/S0
                        if sep/2<TAD:
                            ctcf_bs_time0_predicted[m,i,j,k] += weight[o]*(1/(1+2*c*(1-np.exp(-sep/L))*(sep/2-0.5*(L-(L+sep)*np.exp(-sep/L)))/L)*100*(1-np.exp(-L_sep_ratio)))
                        else:
                            ctcf_bs_time0_predicted[m,i,j,k] += weight[o]*(1/(1+(2*c*(1-c)*(1-np.exp(-sep/L))*(sep/2-0.5*(L-(L+sep)*np.exp(-sep/L)))+c**2*(TAD))/L)*100*(1-np.exp(-L_sep_ratio)))

matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(len(bs_list_sorted),2,figsize=(18, 7*len(bs_list_sorted)))
for k in range(len(bs_list_sorted)):
    submatrix = ctcf_bs_time0[:,:,:,k]
    dim1, dim2, dim3 = submatrix.shape
    ctcf_bs_time0_avg = np.mean(submatrix,axis=0)
    ctcf_bs_time0_sem = stats.sem(submatrix,axis=0)
    annot_success = np.empty((dim2, dim3)).astype(str)
    for i in range(dim2):
        for j in range(dim3):
            p = ctcf_bs_time0_avg[i, j]
            c = ctcf_bs_time0_sem[i, j]
            annot_success[i, j] = '%s\n(+/-%s)' % (str(np.around(p,decimals=1)),str(np.around(c,decimals=1))) 

    submatrix_predicted = ctcf_bs_time0_predicted[:,:,:,k]
    dim1, dim2, dim3 = submatrix_predicted.shape
    ctcf_bs_time0_predicted_avg = np.mean(submatrix_predicted,axis=0)
    annot_theory = np.empty((dim2, dim3)).astype(str)
    for i in range(dim2):
        for j in range(dim3):
            p = ctcf_bs_time0_predicted_avg[i, j]
            annot_theory[i, j] = '%s' % (str(np.around(p,decimals=1)))  

    MSE = np.sum((ctcf_bs_time0_avg-ctcf_bs_time0_predicted_avg)**2)/20
    print(MSE)

    
    sns.heatmap(pd.DataFrame(np.transpose(ctcf_bs_time0_avg)),annot=np.transpose(annot_success),fmt='',xticklabels=proc_list_sorted,yticklabels=sep_list_sorted,cmap= 'Blues_r', vmin=0, vmax=100,cbar_kws={'label': 'Simulated % constrained'},ax=axs[k,1])
    axs[k,1].set_ylabel('separation (kb)')
    axs[k,1].set_xlabel('processivity (kb)')
    axs[k,1].tick_params(direction='out', length=6, width=2)

    sns.heatmap(pd.DataFrame(np.transpose(ctcf_bs_time0_predicted_avg)),annot=np.transpose(annot_theory),fmt='',xticklabels=proc_list_sorted,yticklabels=sep_list_sorted,cmap= 'Blues_r', vmin=0, vmax=100,cbar_kws={'label': 'Predicted % constrained'},ax=axs[k,0])
    axs[k,0].set_ylabel('separation (kb)')
    axs[k,0].set_xlabel('processivity (kb)')
    axs[k,0].set_facecolor("0.8584083044982699")
    axs[k,0].tick_params(direction='out', length=6, width=2)
                                       
                                       
fig.tight_layout(pad=1.5)
plt.savefig('Figures/'+'Theory_vs_Simulation_Pconstrained_v6.pdf',format='pdf')
plt.show()